# Rincón de Práctica R

## MLE usando R


<hr>

### Ejemplo 1.

Sea $\epsilon_i\sim i.i.d.\, N(\mu,\sigma^2)$. 

1. Planear la función log-verosimilitud.
2. Realizar en el computador simulación de 100 datos con $\mathbb{N}(\mu_0,\sigma_0^2)=\mathbb{N}(1.7,3^2)$ y estimación ML.


**Respuesta propuesta.**
Como vimos, la log-likelihood function estará dada por 

$$\ell(\boldsymbol{\theta})=-\frac{n\cdot log(2\pi)}{2}-\frac{n\cdot log(\sigma^2)}{2}-\sum_{i=1}^N{\frac{(\epsilon_i-\mu)^2}{2\cdot \sigma^2}}$$

Así, una propuesta a la parte computacional en R sería la siguiente:

In [11]:
# 1. Simular datos
set.seed(123)

n   <- 100
eps <- rnorm(n, 1.7, 3)

summary(eps)


   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
-5.2275  0.2184  1.8853  1.9712  3.7755  8.2620 

In [17]:
# 2. Escribir la función log-likelihood
logL<- function(theta) {
  mu    <- theta[1]
  sigma <- theta[2]
  return( sum(-log(2*pi*sigma^2)/2 - (eps-mu)^2/(2*sigma^2)) )
}


In [22]:
# 3. Optimizar la log-likelihood
#    Se puede usar optim directamente. Sin embargo, 
#    la libreria maxLik se encarga de computos adicionales 
#    por ejemplo, de la matriz hesiana para los s.e.
library(maxLik)
theta.guess <- c(mu = 1, sigma = 1)
mle         <- maxLik(logLik=logL, start=theta.guess)
summary(mle)


--------------------------------------------
Maximum Likelihood estimation
Newton-Raphson maximisation, 8 iterations
Return code 8: successive function values within relative tolerance limit (reltol)
Log-Likelihood: -242.1305 
2  free parameters
Estimates:
      Estimate Std. error t value  Pr(> t)    
mu      1.9712     0.2724   7.235 4.65e-13 ***
sigma   2.7247     0.1925  14.151  < 2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
--------------------------------------------

</br>
<hr>

### Ejemplo 2. 

**Bajo normalidad, MLE y OLS dan el mismo vector estimado de parámetros para el modelo lineal.** 

Sea,

$$Y=\boldsymbol{X}\boldsymbol{\beta}+u \hspace{0.5cm};\hspace{0.5cm}u_i\sim N(0,\sigma^2)$$

1. Planear la función log-verosimilitud
2. Encontrar $\hat{\boldsymbol{\theta}}_{MLE}=(\hat{\boldsymbol{\beta}},\hat{\sigma}^2)$
3. Encontrar la matriz de información
4. Realizar en el computador simulación de 100 datos para $y_i=\beta_0+\beta_1x_{1i}+\beta_2x_{2i}+u_i$, donde $x_1\sim U[0,10]$, $x_2\sim\chi^2_1$, $u_i\sim\mathbb{N}(0,\sigma^2=3^2)$ y $\boldsymbol{\beta}'=(\beta_0,\beta_1,\beta_2)'=(1,0.75,3.2)$. Realizar estimación ML de los parámetros y comparar con OLS.



**Respuesta sugerida.**

Como vimos, para $\boldsymbol{x}=(1,X_1,x_2)'$ y $u_i=y_i-\boldsymbol{x}_i'\boldsymbol{\beta}$ con $\boldsymbol{\beta}'=(\beta_0,\beta_1,\beta_2)'$ se tiene que

$$\ell(\boldsymbol{\theta})=-\frac{n\cdot log(2\pi\sigma^2)}{2}-\frac{(\boldsymbol{y}-\boldsymbol{X}\boldsymbol{\beta})'(\boldsymbol{y}-\boldsymbol{X}\boldsymbol{\beta})}{2\sigma^2}$$

y a partir de las condiciones de primer orden (_F.O.C._) evaluadas en el estimador ML, se obtiene: 
$$\hat{\boldsymbol{\beta}}_{MLE}=(X'X)^{-1}(X'Y)\hspace{0.5cm},\hspace{0.5cm}\hat{\sigma}^2_{MLE}=(Y-X\hat{\beta})'(Y-X\hat{\beta})\cdot n^{-1}$$ 

Notar que la expresión para $\beta$ corresponde a la misma que se obtiene por OLS.

Además, después de resolver, se obtiene la siguiente matriz de información

$$I(\boldsymbol{\beta}_0)=
\left[\begin{array}{cc}
\frac{1}{\sigma_0^2}(X'X) & 0\\
0 & \frac{N}{2\sigma_0^4}
\end{array}\right]$$

Así, una propuesta para el código de programación en R es la siguiente:

In [26]:
# 1. Simulación de datos
n  <- 100 
s0 <- 3;
b0 <- c(1, 0.75, 3.2);

x1 <- runif(n,0,10);
x2 <- rchisq(n,1);
X  <- cbind(rep(1,n) , x1 , x2);
u  <- rnorm(n,0,s0);

y  <- X%*%b0 + u

In [29]:
# 2. Escribir la función log-likelihood
logLikFun <- function(param) {
    sigma <- param[1]
    b     <- param[-1]
    mu    <- X%*%b
    sum(dnorm(y, mean = mu, sd = sigma, log = TRUE))
}


In [31]:
# 3. Optimizar la log-likelihood
.guess <- c(sigma = 1, beta1 = 1, beta2 = 1, beta3=1)
library(maxLik)
mle     <- maxLik(logLik = logLikFun, start = .guess)
summary(mle)


# Comparación con OLS:
summary(lm(y~X-1))

--------------------------------------------
Maximum Likelihood estimation
Newton-Raphson maximisation, 10 iterations
Return code 8: successive function values within relative tolerance limit (reltol)
Log-Likelihood: -252.6117 
4  free parameters
Estimates:
      Estimate Std. error t value  Pr(> t)    
sigma   3.0258     0.2140  14.137  < 2e-16 ***
beta1   0.7039     0.6417   1.097    0.273    
beta2   0.8101     0.1025   7.907 2.64e-15 ***
beta3   3.1972     0.2185  14.629  < 2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
--------------------------------------------


Call:
lm(formula = y ~ X - 1)

Residuals:
    Min      1Q  Median      3Q     Max 
-7.1401 -2.5676 -0.0981  2.1219  7.7896 

Coefficients:
    Estimate Std. Error t value Pr(>|t|)    
X     0.7039     0.6528   1.078    0.284    
Xx1   0.8101     0.1041   7.779 7.88e-12 ***
Xx2   3.1972     0.2219  14.406  < 2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 3.072 on 97 degrees of freedom
Multiple R-squared:  0.9045,	Adjusted R-squared:  0.9015 
F-statistic: 306.1 on 3 and 97 DF,  p-value: < 2.2e-16
